# Purpose
The goal of this notebook is to solve a regression problem without changing it into a classification problem. 

My previous attempt using the red_wine_quality_predictor turned into a binary classification problem. As such, I would like to practice and demonstrate my ability to solve regression problems without turning them into classification problem. 

# 6 Key steps 

We will follow and adapt the 6 key steps of Data analysis to this project.

6 Key steps: 

1) Ask 
- Define what the project would look like 
- Define what would qualify as successful result

2) Prepare 
- Build a timeline and determine how much time is needed
- Determine what data is needed to achieve a successful result

3) Process
- Collect, store, manage and protect the data.
- Processed the data by cleaning it to make sure it was complete, correct, relevant, and free of errors and outliers.

4) Analyze
- Experiment with different models and hyperparameters to meet criterias of success 
- Analyze data

5) Share / Visualize 
- Process and visualize data to share / make meaningful deductions from 

6) Act
- Determine how to implement changes and take actions based on the findings

# 1) Ask 
---

## Define what the project would look like 
* The goal of this notebook is to solve a regression problem without changing it into a classification problem 
* Therefore the goal will be utilizing the features( `longitude`,	`latitude`,	`housing_median_age`, `total_rooms`,	`total_bedrooms`, `population`, `households`, `median_income`) in the `california_housing` dataset, accuracrately predict the `median_house_value`

* A possible variation to experiment with is using `longitude`,	`latitude`,	`housing_median_age`, `total_rooms`,	`total_bedrooms`, `population`, `households`,  and `median_house_value` to determine what `median_income` would be needed to live there.



## Define what would qualify as a successful result
* We will aim for an accuracy of **>95%**

# 2) Prepare 
---
## Build a timeline and determine how much time is needed
* The estimated timeline will be a working week (5 day) sprint 

## Determine what data is needed to achieve a successful result
* We will be utilzing the sample data from google collab which has been downloaded and added to github : https://github.com/Kirlewn/Machine-Learning-Projects/tree/main/Data/california_housing_data 


In [ ]:
# import standard expected libraries
import pandas as pd
import numpy as np 
import tensorflow as tf
import sklearn as sk 
import matplotlib.pyplot as plt
import seaborn as sns 

# 3) Process
---


##  Collect data

In [1]:
# the data is already pre-split into train and test so let's begin with importing the train data
train_df = pd.read_csv("https://raw.githubusercontent.com/Kirlewn/Machine-Learning-Projects/main/Data/california_housing_data/california_housing_train.csv")

train_df.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value
0,-122.05,37.37,27.0,3885.0,661.0,1537.0,606.0,6.6085,344700.0
1,-118.30,34.26,43.0,1510.0,310.0,809.0,277.0,3.5990,176500.0
2,-117.81,33.78,27.0,3589.0,507.0,1484.0,495.0,5.7934,270500.0
3,-118.36,33.82,28.0,67.0,15.0,49.0,11.0,6.1359,330000.0
4,-119.67,36.33,19.0,1241.0,244.0,850.0,237.0,2.9375,81700.0


In [ ]:
# let's also get the test 
test_df = pd.read_csv("https://raw.githubusercontent.com/Kirlewn/Machine-Learning-Projects/main/Data/california_housing_data/california_housing_test.csv")

test_df.head()

## Data Exploration
Determine if there are any unusuable or strange data. 
This includes outliers, missing values, mismatched data types (an column with ints having a string input), etc. 

## Preprocess

Processed the data by cleaning it to make sure it was complete, correct, relevant, and free of errors and outliers.

# 4) Analyze
---

Experiment with different models and hyperparameters to meet criterias of success. 

Outlier Experiments

Model Experiments



Analyze data